David Shnaiderov
Shahar Rapp

In [2]:
from qiskit.quantum_info.operators import Operator
import qiskit.quantum_info as qi
import numpy as np
from qiskit.circuit.library import TwoLocal
from qiskit.circuit import QuantumCircuit, Parameter

from qiskit_algorithms import VQE

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_algorithms.optimizers import COBYLA

from qiskit_ibm_runtime.fake_provider import FakeLimaV2
from qiskit_aer.noise import NoiseModel

import matplotlib.pyplot as plt

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


In [3]:
g_0 = -0.4804
g_1 = 0.3435
g_2 = -0.4347
g_3 = 0.5716
g_4 = 0.0910
g_5 = 0.0910

paulis = ["II", "ZI", "IZ", "ZZ", "YY", "XX"]
coeffs = [g_0, g_1, g_2, g_3, g_4, g_5]

H2 = sum(coeff * Operator(qi.SparsePauliOp(pauli)) for coeff, pauli in zip(coeffs, paulis))

print(H2)


Operator([[ 1.11022302e-16+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            0.00000000e+00+0.j],
          [ 0.00000000e+00+0.j, -2.73800000e-01+0.j,  1.82000000e-01+0.j,
            0.00000000e+00+0.j],
          [ 0.00000000e+00+0.j,  1.82000000e-01+0.j, -1.83020000e+00+0.j,
            0.00000000e+00+0.j],
          [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
            1.82400000e-01+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))


Exact solution

In [4]:
# Compute the eigenvalues of H2
eigen_values = np.linalg.eigvalsh(H2)
min_eigenvalue = min(eigen_values)
print("Exact result:", min_eigenvalue)
nuclear_repulsion = 0.7055696146
hydrogen_atom_min_eigenvalue = min_eigenvalue + nuclear_repulsion
print("Plus chemistry:", hydrogen_atom_min_eigenvalue)

Exact result: -1.851199124123644
Plus chemistry: -1.1456295095236442


### Ideal simulator:

In [5]:
shots = 10000

num_qubits = 2
layers = 1
ansatz = TwoLocal(num_qubits, 'ry', 'cx', reps=layers, insert_barriers=False, skip_final_rotation_layer=True)
optimizer = COBYLA(maxiter=1000, tol=0.00001)

ideal_estimator = AerEstimator(
    run_options={
        "shots": shots})
counts = []
values = []


def show_results(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    print(f"VQE on Aer qasm simulator (noisy): {mean:.7f}")
hamiltonian = qi.SparsePauliOp.from_list(list(zip(paulis, coeffs)))
vqe = VQE(ideal_estimator, ansatz, optimizer, callback=show_results)
result = vqe.compute_minimum_eigenvalue(operator=hamiltonian)
print(result)

VQE on Aer qasm simulator (noisy): -0.8554973
VQE on Aer qasm simulator (noisy): -0.2857076
VQE on Aer qasm simulator (noisy): -0.8629798
VQE on Aer qasm simulator (noisy): -1.5526768
VQE on Aer qasm simulator (noisy): -1.7128595
VQE on Aer qasm simulator (noisy): -0.9591827
VQE on Aer qasm simulator (noisy): -1.6509267
VQE on Aer qasm simulator (noisy): -1.4497091
VQE on Aer qasm simulator (noisy): -1.7498528
VQE on Aer qasm simulator (noisy): -1.8337838
VQE on Aer qasm simulator (noisy): -1.8493383
VQE on Aer qasm simulator (noisy): -1.8359916
VQE on Aer qasm simulator (noisy): -1.8085974
VQE on Aer qasm simulator (noisy): -1.8325724
VQE on Aer qasm simulator (noisy): -1.8477574
VQE on Aer qasm simulator (noisy): -1.8519360
VQE on Aer qasm simulator (noisy): -1.8435129
VQE on Aer qasm simulator (noisy): -1.8489370
VQE on Aer qasm simulator (noisy): -1.8481556
VQE on Aer qasm simulator (noisy): -1.8445048
VQE on Aer qasm simulator (noisy): -1.8481954
VQE on Aer qasm simulator (noisy):

### Noisy simulator:

In [6]:
fake_lima = FakeLimaV2()
noise_model = NoiseModel.from_backend(fake_lima)
noisy_estimator = AerEstimator(
    backend_options={
        "method": "density_matrix",
        "noise_model": noise_model,
    },
)

vqe = VQE(noisy_estimator, ansatz, optimizer, callback=show_results)
result = vqe.compute_minimum_eigenvalue(operator=hamiltonian)


print(result)



VQE on Aer qasm simulator (noisy): -1.3524824
VQE on Aer qasm simulator (noisy): -0.9832834
VQE on Aer qasm simulator (noisy): -1.7287039
VQE on Aer qasm simulator (noisy): -1.4632139
VQE on Aer qasm simulator (noisy): -1.5324639
VQE on Aer qasm simulator (noisy): -1.7650443
VQE on Aer qasm simulator (noisy): -1.7080168
VQE on Aer qasm simulator (noisy): -1.6628488
VQE on Aer qasm simulator (noisy): -1.6982285
VQE on Aer qasm simulator (noisy): -1.7468533
VQE on Aer qasm simulator (noisy): -1.7235768
VQE on Aer qasm simulator (noisy): -1.7006988
VQE on Aer qasm simulator (noisy): -1.7206557
VQE on Aer qasm simulator (noisy): -1.7512115
VQE on Aer qasm simulator (noisy): -1.7414566
VQE on Aer qasm simulator (noisy): -1.7265691
VQE on Aer qasm simulator (noisy): -1.7373391
VQE on Aer qasm simulator (noisy): -1.7235064
VQE on Aer qasm simulator (noisy): -1.7173393
VQE on Aer qasm simulator (noisy): -1.7350104
VQE on Aer qasm simulator (noisy): -1.7366346
VQE on Aer qasm simulator (noisy):

In [7]:
# IBM account
token = '5a27fdfe6f926662f4c32c380dd6e6ef6a239da857c3b024ca5b8113c460b7a4f1272bca9594dc04519ffa8f10d4dd99b6d9c4142d7da578577dc80a783c0d0c'
service = QiskitRuntimeService(channel="ibm_quantum", token=token)
# Save an IBM Quantum account and set it as your default account.
QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token=token,
    set_as_default=True,
    overwrite=True,)

In [9]:
from qiskit_ibm_runtime import Session,EstimatorV2 as Estimator

In [10]:
shots = 10000
backend = service.least_busy(min_num_qubits=num_qubits, operational=True)

pm = generate_preset_pass_manager(target=backend.target, optimization_level=2)
transplie_circuit = pm.run(ansatz)
observables = hamiltonian.apply_layout(transplie_circuit.layout)
theta = [0, 1]
# Start a runtime session
with Session(service=service, backend=backend) as session:
    # Set up the Estimator
    estimator = Estimator(session)
    estimator.options.default_shots  = shots
    estimator.options.resilience_level = 0  #No error mitigation is applied to the user program.

    job = estimator.run([(transplie_circuit,observables, theta)])

    res = job.result()[0]
    print(f"min energy on real quantum machine: {res.data.evs}")

    estimator.options.resilience_level = 1  #with error mitigation
    job2 = estimator.run([(transplie_circuit,observables, theta)])

    res2 = job2.result()[0]
    print(f"min energy on real quantum machine with error mitigation: {res2.data.evs}")


min energy on real quantum machine: -0.5676677400000001
min energy on real quantum machine with error mitigation: -0.39885877981370677


### Analyze results
Exact solution: -1.851199124123644
Ideal simulation: -1.85155
Noisy simulation: -1.73709
Real hardware without error mitigation: -0.56766774
Real hardware with error mitigation: -0.398858

The comparison of these results demonstrates the challenges in achieving accurate quantum computations on real hardware. Noise and hardware imperfections significantly affect the results, but error mitigation techniques can help reduce these errors, although not entirely eliminating them. The closer alignment of the ideal simulation with the exact solution confirms that the algorithm is fundamentally working.